In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [7]:
img_3 = np.zeros([720,1080,3],dtype=np.uint8)
img_3_align = np.zeros([720,1080,3],dtype=np.uint8)
img_3.fill(255)
img_3_align.fill(255)
cv2.namedWindow("ws5")
cv2.namedWindow("align image")

In [8]:
def align_image(reference_image = None, align_image = None,TOP_MATCH_COUNT = 0, MIN_MATCH_COUNT = 4):
    print(TOP_MATCH_COUNT)
    reference_image_grey = cv2.cvtColor(reference_image, cv2.COLOR_BGR2GRAY)
    align_image_grey = cv2.cvtColor(align_image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    kp_ref, des_ref = sift.detectAndCompute(reference_image_grey,None)
    kp_align, des_align = sift.detectAndCompute(align_image_grey, None)
    # kp_detect_image_ref=cv2.drawKeypoints(reference_image_grey,kp_ref,reference_image)
    # kp_detect_image_align = cv2.drawKeypoints(align_image_grey,kp_align,align_image)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des_ref,des_align, k = 2)
    print(des_ref)
    print(type(des_align))
    #print(matches)
    # Apply ratio test
    all = []
    good = []
    all_mask = []
    for m,n in matches:
        if m.distance < .75*n.distance:
            good.append(m)
        all.append(m)
    for i in range(len(all)):
        all_mask.append(1)
    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp_ref[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp_align[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        M, mask = cv2.findHomography(dst_pts,src_pts, cv2.RANSAC)
        matchesMask = mask.ravel().tolist()
        print(M)
        h,w = reference_image_grey.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,M)
        #aligned_image = cv2.polylines(align_image,[np.int32(dst)],True,255,3, cv2.LINE_AA)
        aligned_image = cv2.warpPerspective(align_image, M, (w, h))
 
    else:
        print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
        matchesMask = None
    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
    all_params = dict(matchColor = (255,0,0), # draw matches in red color
                   singlePointColor = None,
                   matchesMask = all_mask[:TOP_MATCH_COUNT+4] if TOP_MATCH_COUNT > 0 else all_mask[:], 
                   flags = 2)
    #height, width, channels = reference_image.shape
    #Ransac_align = cv2.warpPerspective(align_image, M, (width, height))
    img_match = cv2.drawMatches(reference_image,kp_ref,align_image,kp_align,good,None,**draw_params)
    #plt.imshow(img_match),plt.show()
    img_all_match = cv2.drawMatches(reference_image,kp_ref,align_image,kp_align,all[:TOP_MATCH_COUNT+4] if TOP_MATCH_COUNT > 0 else all[:],None,**all_params)
    #plt.imshow(img_all_match),plt.show()
    return img_match, img_all_match, aligned_image


In [9]:
while(True):
    cv2.imshow('ws5', img_3)
    cv2.imshow('align image', img_3_align)
    k = cv2.waitKey(0)
    if k == ord('f'):
        img_3 = cv2.imread('cat.jpg')
        img_3_align = cv2.imread('cat_a.jpg')
        img_match_ransac, img_match_all, aligned_image= align_image(img_3, img_3_align,23)
    if k == ord('l'): # load
        reference_location = input('Paste the reference image location:')
        img_3 = cv2.imread(reference_location)
        align_location = input('Paste the align image location:')
        img_3_align = cv2.imread(align_location)
        top_count = int(input('How many line you want to show in all key point'))
        blank = np.zeros(shape=(img_3.shape[0], img_3.shape[1]))
        blank.fill(255)
        blank_align = np.zeros(shape=(img_3_align.shape[0], img_3_align.shape[1]))
        blank_align.fill(255)
        cv2.imshow('ws5', img_3)
    if k == 13: # enter key
        img_match_ransac, img_match_all, aligned_image= align_image(img_3, img_3_align,top_count)
        cv2.imshow('Ransac_align', aligned_image)
        cv2.imshow('img_match_ransac', img_match_ransac)
        cv2.imshow('img_match_all', img_match_all)
    if k == ord('c'): 
        img_3 = blank.copy()
    if k == ord('v'): 
        img_3_align = blank.copy()
    if (k == ord('q')): # quit
        cv2.destroyAllWindows()
        break

23
[[  4.   3.   5. ...   0.   0.   0.]
 [  0.   0.   0. ...  87. 140.   5.]
 [  0.   0.   0. ...   1.   0.   0.]
 ...
 [ 22.   0.   0. ...   0.   0.  20.]
 [  7.   1.   0. ...   0.   0.   2.]
 [  1.   2.   1. ...   9.  37.  35.]]
<class 'numpy.ndarray'>
[[-9.99461162e-01 -1.15767069e+00  9.04045883e+02]
 [-4.19045869e-03 -9.71410684e-01  3.58697549e+02]
 [-1.56443809e-05  7.46157961e-05  1.00000000e+00]]
